## Iniciar Contexto do Spark

In [1]:
import os
import time
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages datastax:spark-cassandra-connector:2.0.6-s_2.11 --conf spark.cassandra.connection.host=172.17.0.2 pyspark-shell'

conf = SparkConf()
conf.setMaster("local[*]")
conf.setAppName("Cassandra crud test")

sc = SparkContext(conf=conf)
sql = SQLContext(sc)

events = sql.read.format("org.apache.spark.sql.cassandra").load(keyspace="sea", table="events")

events.registerTempTable("events")

print("Contexto Iniciado!")

Contexto Iniciado!


### Histograma - Eventos x Estado

In [ ]:
events_state = sql.sql("select * from (select count(0) total, regexp_replace(state,'State of ','') rep_state, category_name from events where country = 'Brazil' and state is not null and category_name is not null and category_name not like 'Acidente' group by rep_state, category_name) order by total desc")
events_state.show()

### Linhas Eventos por Mês - Ultimos 30 dias

In [ ]:
events_last_month = sql.sql("select * from (select count(0) total, from_unixtime(timestamp_ms/1000,'yyyy-MM-dd') event_date, category_name from events where country = 'Brazil' and category_name is not null and category_name not like 'Acidente' group by event_date, category_name) order by event_date asc")
events_last_month.show()

### Top 10 - Cidades com mais eventos

In [ ]:
top_10_cities = sql.sql("select * from (select count(0) total, city, regexp_replace(state,'State of ','') state_name, category_name from events where country = 'Brazil' and city is not null and category_name is not null and category_name not like 'Acidente' group by city, state_name, category_name) order by total desc")
top_10_cities.show()

### Mapa Interativo - Latitude Longitude

In [ ]:
event_map = sql.sql("select from_unixtime(timestamp_ms/1000,'yyyy-MM-dd HH:mm') event_time, latitude, longitude, category_name from events where country = 'Brazil' and latitude is not null and longitude is not null and category_name is not null and category_name not like 'Acidente' and from_unixtime(timestamp_ms/1000,'yyyy-MM-dd') = '2017-12-29' order by timestamp_ms")
event_map.show()

In [3]:
sql.sql("select category_name as label, category_name as value from events where category_name is not null group by category_name order by category_name").show()

+------------+------------+
|       label|       value|
+------------+------------+
|    Arrastão|    Arrastão|
|     Assalto|     Assalto|
|Manifestação|Manifestação|
|       Morte|       Morte|
|       Roubo|       Roubo|
|    Tiroteio|    Tiroteio|
+------------+------------+

